<a href="https://colab.research.google.com/github/tanishqnavlakhe/FMML_Project_and_Labs/blob/main/Copy_of_Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets #it is used for classification,regretion ,clusturing
# set randomseed
rng = np.random.default_rng(seed=42) ##this is done through numpy library through this algorithum we can generate a random number which can be predicted and reproduced
#through seed we provide a initial value to random seed generator if we provide a same seed again the random seed generator will generate the same sequence of random number every time we run a program

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 #print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 #print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify # in this we are converting a target value from data set to certain datatype which is an integer
 print(dataset.data.shape) #this line of code is used to print dimention of dataset like how many row vs column. so dataset have 20640 sample and 8 feature
 print(dataset.target.shape)# total target value or sample are 20640

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])
(20640, 8)
(20640,)


<ipython-input-6-e74f7234b7be>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3.  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph usinghtml#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%


answer 3

**For Nearest Neighbor Classifier:**

1. **Training Set Size:** In general, you should allocate a significant portion of your data for the training set. A common split is around 70-80% of the data for training.

2. **Validation Set Size:** For validation, you can start with a 20-30% split. So, a 70-30 or 80-20 split between training and validation can be a reasonable starting point.

**For Random Classifier:**

1. **Training Set Size:** Similar to the nearest neighbor classifier, allocate a significant portion (around 70-80%) of your data for the training set.

2. **Validation Set Size:** Since a random classifier doesn't learn patterns in the data, you can allocate a smaller portion for validation compared to the nearest neighbor approach. A 10-20% split for validation can be a reasonable starting point.

```python
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Experiment with different validation set sizes
validation_sizes = [0.1, 0.2, 0.3, 0.4]
accuracies = []

for val_size in validation_sizes:
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=val_size, random_state=42)
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    accuracies.append(accuracy)

# Plot the results
plt.plot(validation_sizes, accuracies, marker='o')
plt.xlabel("Validation Set Size")
plt.ylabel("Accuracy")
plt.title("Validation Set Size vs. Accuracy")
plt.grid(True)
plt.show()
```



answer 2


The size of the training and validation sets can indeed affect how well we can predict the accuracy on the test set using the validation set. Here's how:

**1. Large Training Set and Small Validation Set:**
   - When you have a large training set and a small validation set, your model has more data to learn from during training. This can be beneficial because it's exposed to a wider range of examples and can potentially capture complex patterns in the data.
   - However, if the validation set is too small, it might not provide a reliable estimate of how well your model generalizes to unseen data. The validation set's performance might be highly variable and may not be a good indicator of how the model will perform on the test set.

**2. Small Training Set and Large Validation Set:**
   - With a small training set and a large validation set, your model has less data to learn from, which can lead to underfitting. It might not capture complex patterns well because it hasn't seen enough examples.
   - A large validation set can provide a more stable estimate of your model's performance, but it may not be able to fully assess the model's capability to generalize, especially if the training set is small.

**3. Balanced Training and Validation Sets:**
   - Ideally, you want a balance between the training and validation set sizes. A common practice is to use a significant portion of your data for training and a smaller, but still representative, portion for validation.
   - A balanced approach allows your model to learn from a sufficiently large training set while still having a validation set that provides a reasonably reliable estimate of generalization performance.
   - Cross-validation, where the data is divided into multiple subsets (folds) for training and validation, is another technique to achieve a balance and reduce variability in the validation results.



answer 1

In machine learning, the size of the validation set, relative to the training set, can have a significant impact on the evaluation and performance of your models. Let's explore how increasing or reducing the percentage of the validation set can affect model evaluation:

**1. Increasing the Percentage of the Validation Set:**

- **Pros:**
  - With a larger validation set, you have more data to evaluate your model's performance. This can provide a more reliable estimate of how well your model generalizes to new, unseen data.
  - It can help detect issues like overfitting, where the model performs well on the training data but poorly on new data. A larger validation set is more likely to expose such issues.

- **Cons:**
  - As you increase the size of the validation set, you reduce the amount of data available for training your model. This can be problematic if your training dataset is small to begin with because it may lead to underfitting, where your model doesn't learn complex patterns effectively.
  - Training on a smaller portion of the data might also make it harder for the model to capture the full diversity of the dataset, potentially limiting its performance.

**2. Reducing the Percentage of the Validation Set:**

- **Pros:**
  - A smaller validation set means you have more data available for training. This can be advantageous when working with limited data because your model has more examples to learn from.
  - Training on a larger portion of the data can enable your model to capture more nuanced patterns and potentially lead to better performance on the training data.

- **Cons:**
  - With a smaller validation set, the evaluation of your model's performance may be less reliable. The estimate of how well your model generalizes may have higher variance, meaning it can be influenced by random variations in the data.
  - It may be more challenging to detect overfitting with a smaller validation set, as the model may appear to perform well on it even if it's not generalizing well to new data.



## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


answer 1


**Title: The Importance of Averaging Validation Accuracy in Cross-Validation**

In machine learning experiments, the choice of a single validation set can sometimes introduce variability in the results. Averaging the validation accuracy across multiple splits, a technique known as cross-validation, offers several advantages and contributes to more consistent and reliable model evaluation.

1. **Reduced Variability:** When we use a single fixed validation set, the model's performance can be influenced by the specific subset of data chosen for validation. This can lead to results that may not fully represent the model's true capability. Cross-validation mitigates this issue by systematically rotating through different subsets of the data, ensuring that the model is evaluated on multiple partitions. Averaging these results provides a more stable estimate of performance.

2. **Better Generalization Assessment:** Cross-validation provides a more robust estimate of a model's ability to generalize to unseen data. By training and validating the model on different subsets of the data, it ensures that the model's performance is evaluated across a range of data patterns and variations. This makes the assessment of generalization more accurate.

3. **Optimal Hyperparameter Tuning:** Cross-validation is invaluable when tuning hyperparameters in machine learning models. It allows us to evaluate how different parameter settings impact performance across multiple data subsets. Averaging the results ensures that our hyperparameter choices are less likely to be influenced by random fluctuations in a single validation split.

4. **Improved Model Selection:** In cases where we're comparing multiple models, cross-validation provides a fair and consistent basis for comparison. It ensures that each model is evaluated on the same data partitions, reducing the potential for bias in model selection.

5. **Enhanced Confidence:** By averaging validation results over multiple splits, we can calculate not only a point estimate of model performance but also measures of confidence, such as standard deviation or confidence intervals. This information is valuable for understanding the stability and reliability of our results.

In conclusion, averaging the validation accuracy across multiple splits, as achieved through techniques like k-fold cross-validation, is a fundamental practice in machine learning. It helps reduce variability, provides a more accurate assessment of generalization, aids in hyperparameter tuning and model selection, and enhances our confidence in the reported results. For these reasons, it is considered a best practice in the field of machine learning and data science.

answer 2


**Does it give a more accurate estimate of test accuracy?**

   Cross-validation, particularly when paired with appropriate techniques like k-fold cross-validation, provides a more accurate estimate of a model's generalization performance compared to a single fixed validation set. Here's why:

   - **Better Generalization Assessment:** Cross-validation assesses how well your model generalizes to unseen data by evaluating it on multiple partitions of the dataset. This comprehensive evaluation allows you to obtain a more reliable estimate of how your model is likely to perform on new, unseen data. It provides a more accurate reflection of your model's true generalization ability.

   - **Reduced Bias:** Averaging the results of cross-validation reduces the risk of bias associated with a single validation set. Bias can occur when a particular validation set happens to be either easier or harder than average, leading to an inaccurate estimate of test accuracy. Cross-validation mitigates this bias by systematically considering multiple validation partitions.

   - **Robust Hyperparameter Tuning:** Cross-validation is essential for hyperparameter tuning, allowing you to evaluate different hyperparameter settings across multiple data partitions. Averaging these evaluations helps you make more informed decisions about hyperparameters and ensures that your choices are less likely to be influenced by chance.


answer3

The number of iterations in machine learning algorithms can have an impact on the estimate of a model's performance and, in some cases, can lead to a better estimate with higher iterations. However, the relationship between the number of iterations and the quality of the estimate depends on various factors, including the specific algorithm, the dataset, and the problem you're trying to solve. Here's a more detailed explanation:

**1. Effect of Number of Iterations:**

- In many machine learning algorithms, such as those involving optimization (e.g., gradient descent), the number of iterations often determines how well the model converges to an optimal solution. Convergence refers to the point at which the algorithm stops updating the model's parameters because it has reached a reasonably good solution.

- With a small number of iterations, the algorithm may not have sufficient time to converge to an optimal solution. As a result, the model's performance may not be as good as it could be.

- Conversely, increasing the number of iterations allows the algorithm more time to fine-tune the model's parameters. In some cases, this can lead to better performance and a more accurate estimate of the model's capabilities.

**2. Diminishing Returns:**

- It's important to note that the relationship between the number of iterations and performance is often subject to diminishing returns. In other words, as you increase the number of iterations, the improvements in performance tend to become smaller and smaller.

- There's usually an optimal point at which further increasing the number of iterations doesn't significantly improve the estimate of the model's performance. Beyond this point, you might experience overfitting, where the model starts to fit the training data noise rather than the underlying patterns.

**3. Trade-off:**

- The number of iterations represents a trade-off between computation time and performance. Running a model for too many iterations can be computationally expensive, especially for large datasets and complex models.

- Therefore, it's often necessary to find a balance between computational resources and model performance. This is typically achieved through hyperparameter tuning, where you experiment with different settings for the number of iterations and choose the one that provides the best trade-off between training time and model accuracy.

**4. Algorithm-Specific Considerations:**

- The impact of the number of iterations can vary significantly between different machine learning algorithms. Some algorithms may require a large number of iterations to converge, while others may converge quickly.

- Additionally, some algorithms may benefit from early stopping, a technique where the training process is halted if performance on a validation set doesn't improve after a certain number of iterations. This prevents the model from overfitting.

In conclusion, the number of iterations in machine learning algorithms can influence the estimate of a model's performance, but it's not a guarantee that more iterations will always result in a better estimate. Finding the right number of iterations often requires experimentation, consideration of computational constraints, and understanding the behavior of the specific algorithm you're using.